In [1]:
import sys, os, glob
import argparse
import pandas as pd
import numpy as np

fits_summary_path = "/Users/faerest/Desktop/pantheon_test_file/test1/fits_summary.csv"
brightest_galaxy_path = "/Users/faerest/Desktop/pantheon_test_file/test1/brightest_galaxy.csv"
all_ds9_out_path = "./ds9_cmds_all.csv"
supernova_names = ["2001ah"]

In [ ]:
fits_summary = pd.read_table(fits_summary_path,sep=',')
print(fits_summary.to_string())

sn_ix = np.where(fits_summary["SNID"] == "2001ah")[0]
print(sn_ix)

       SNID                                                                                                                                                 filename                 Date Telescope Instrument      Filter         CD1_1         CD1_2         CD2_1         CD2_2
0    1997bq                                pantheon_data_folder/1997bq/mastDownload/HST/hst_16234_70_wfc3_ir_f160w_ieel70/hst_16234_70_wfc3_ir_f160w_ieel70_drz.fits           2024-11-09       HST       WFC3       F160W -3.562500e-05 -7.487909e-22 -7.171354e-22  3.562500e-05
1    1997bq                                                                                pantheon_data_folder/1997bq/mastDownload/HST/idgg31020/idgg31020_drz.fits           2025-04-07       HST       WFC3       F160W  3.115917e-05  1.726983e-05  1.726983e-05 -3.115917e-05
2    1997bq                                                                                pantheon_data_folder/1997bq/mastDownload/HST/idgg3b020/idgg3b020_drz.fits           

In [3]:
brightest_galaxy = pd.read_table(brightest_galaxy_path,sep=',')

print(brightest_galaxy)
print(f'{brightest_galaxy_path}')

       SNID                               File_key  Galaxy_Key   RA_Galaxy  \
0    1997bq  hst_15145_33_wfc3_ir_f160w_idgg33_drz           1  154.223031   
1    1997bq                          idgg33020_drz           2  154.222762   
2    1997bq  hst_15145_39_wfc3_ir_f160w_idgg39_drz           3  154.223238   
3    1997bq                          idgg31020_drz           4  154.223045   
4    1997bq  hst_15145_37_wfc3_ir_f160w_idgg37_drz           5  154.223224   
..      ...                                    ...         ...         ...   
633   2002G  hst_15145_31_wfc3_ir_f160w_idgg31_drz         634  154.222991   
634   2002G  hst_16234_70_wfc3_ir_f160w_ieel70_drz         635  154.203397   
635   2002G  hst_15145_35_wfc3_ir_f160w_idgg35_drz         636  154.223017   
636  2003cg         HST_7331_98_NIC_NIC2_F160W_drz         637  153.562681   
637  2003cg                          n3zd98010_mos         638  153.562535   

     Dec_Galaxy  Position_Angle  Major_Axis  Minor_Axis      Ma

In [4]:
pd.set_option('display.max_colwidth', None)
fits_summary['filenameshort'] = fits_summary['filename'].str.replace('.*\/','',regex=True)
# print(fits_summary['filenameshort'].to_string())

# for column_name in brightest_galaxy.columns:
#     if column_name not in fits_summary.columns:
#         fits_summary[column_name] = np.nan

# print(fits_summary.head().to_string())
# print(brightest_galaxy.head().to_string())

In [ ]:
for sn_name in supernova_names:
    print("SN name: ",sn_name)

    sn_ix_bg = brightest_galaxy[brightest_galaxy["SNID"] == sn_name].index.values
    print("Brightest galaxy indices: ",sn_ix_bg)

    sn_ix = np.where(fits_summary["SNID"] == sn_name)[0]
    print("Fits summary indices: ",sn_ix)

    for index in sn_ix_bg:
        # print(brightest_galaxy.loc[index_bg,['SNID','File_key']].to_string())
        filename = brightest_galaxy.loc[index,'File_key']
        # print('Filename: ',filename) 

        filename_ix_fs = sn_ix[np.where(fits_summary.loc[sn_ix, "filenameshort"] == filename+".fits")[0]]
        # print(filename_ix_fs)
        
        # print(fits_summary.loc[filename_ix_fs].to_string())
        # print(brightest_galaxy.loc[[index_bg]].to_string()) 

        if len(filename_ix_fs)>1:
            raise RuntimeError(f"filename_ix_fs is returning multiple matches for filenameshort in fits_summary: {filename_ix_fs}")

        for column_name in brightest_galaxy.columns:
            # print(column_name)
            if column_name not in fits_summary.columns:
                fits_summary[column_name] = np.nan
            # print(brightest_galaxy.at[index_bg, column_name])
            fits_summary.at[filename_ix_fs[0], column_name] = brightest_galaxy.at[index, column_name]

    print(fits_summary.loc[sn_ix].to_string())

SN name:  2001ah
Brightest galaxy indices:  [398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415
 416 417 418 419 420 421 422 423 424]
Fits summary indices:  [498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515
 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532]
       SNID                                                                                                                                                 filename        Date Telescope Instrument     Filter     CD1_1         CD1_2         CD2_1     CD2_2                                              filenameshort                                              File_key  Galaxy_Key   RA_Galaxy  Dec_Galaxy  Position_Angle  Major_Axis  Minor_Axis      Mag  Mag_Error          Flux  Flux_Error       CXX       CXY        CYY
498  2001ah                                pantheon_data_folder/2001ah/mastDownload/HST/hst_16075_01_wfc3_ir_f110w_ie9801/hst_16075_01_wfc3_ir_f110w_ie9801_

In [ ]:
# ds9:
#ellipse(186.8194200,9.4313379,10.000",30.000",44.999994) # color=red width=4
#ellipse(RA, Dec, minor axis lenght, major axis length, Poisition Angle PA in deg)
def make_region_command_for_position(ra,dec,position_angle,major_axis,minor_axis,name=None,symbol='circle',color='red',width=4,size='10"'):
    print(f'Making region command: RA_Galaxy {ra}, Dec_Galaxy {dec}, name {name}, Position_Angle {position_angle}, Major_Axis {major_axis}, Minor_Axis {minor_axis}')
    name='{'+f'{name}'+'}'
    s = f'circle({ra},{dec},{size},{position_angle},{major_axis},{minor_axis}) # color={color} width={width} text={name}'
    print(s)
    return(s)

def get_region_info(cd_matrix, cov_matrix):
    # 3. World-space covariance matrix (deg²)
    cov_world = cd_matrix @ cov_matrix @ cd_matrix.T

    # 4. Eigen-decomposition
    eigvals, eigvecs = np.linalg.eigh(cov_world)  # always returns sorted values

    # Convert eigenvalues (deg²) to axis lengths (arcsec)
    # Note: 1 deg = 3600 arcsec, so variance in deg² -> stddev in arcsec = sqrt(val) * 3600
    a_arcsec = 3600 * np.sqrt(eigvals[1])  # major axis
    b_arcsec = 3600 * np.sqrt(eigvals[0])  # minor axis

    # 5. Position angle (in degrees, from North to East)
    vec = eigvecs[:, 1]  # eigenvector of major axis
    theta_rad = np.arctan2(vec[0], vec[1])  # careful: note the order (RA, Dec)
    theta_deg = np.degrees(theta_rad)

    # Normalize angle to 0–180
    theta_deg = (theta_deg + 360) % 180

    return theta_deg, a_arcsec, b_arcsec

def save_region_file(filepath, output):
    print(f'Writing region file to {filepath}')
    with open(filepath, "w") as file:
        for line in output:
            file.write(str(line) + "\n")
    file.close()	

for sn_name in supernova_names:
    print("SN name: ",sn_name)

    sn_ix = np.where(fits_summary["SNID"] == sn_name)[0]

    ra_galaxy = None
    dec_galaxy = None
    
    for index in sn_ix:
        print("Index:", index)
        print("Row: \n",fits_summary.iloc[[index]].to_string())

        file_key = fits_summary.at[index, "File_key"]
        if isinstance(file_key, float) and np.isnan(file_key):
            continue

        if ra_galaxy is None or dec_galaxy is None:
            ra_galaxy = fits_summary.at[index, "RA_Galaxy"]
            dec_galaxy = fits_summary.at[index, "Dec_Galaxy"]
            print(f"Setting RA and Dec: {ra_galaxy}, {dec_galaxy}")
        
        print(f"RA and Dec: {ra_galaxy}, {dec_galaxy}")
        
        # 1. Pixel-space covariance matrix
        cov_matrix = np.array([
            [fits_summary.at[index, "CXX"], fits_summary.at[index, "CXY"]],
            [fits_summary.at[index, "CXY"], fits_summary.at[index, "CYY"]]
        ])

        # 2. CD matrix (deg/pixel)
        cd_matrix = np.array([
            [fits_summary.at[index, "CD1_1"], fits_summary.at[index, "CD1_2"]],
            [fits_summary.at[index, "CD2_1"], fits_summary.at[index, "CD2_2"]]
        ])

        theta_deg, a_arcsec, b_arcsec = get_region_info(cd_matrix, cov_matrix)

        cmd = make_region_command_for_position(ra_galaxy, dec_galaxy, theta_deg, a_arcsec, b_arcsec, name=sn_name)

        # make region file
        output = ['global color=blue dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1']
        output.append('fk5')
        output.append(cmd)

        print("OUTPUT:")
        for item in output:
            print(item)

        filename = f"{file_key}_region_file.reg"
        filedir = f"./region_files"
        if not os.path.exists(filedir):
            os.mkdir(filedir)
        filepath = f"{filedir}/{filename}"
        save_region_file(filepath, output)
        
        break

SN name:  2001ah
Index: 498
Row: 
        SNID                                                                                                                   filename        Date Telescope Instrument Filter     CD1_1         CD1_2         CD2_1     CD2_2                               filenameshort                               File_key  Galaxy_Key   RA_Galaxy  Dec_Galaxy  Position_Angle  Major_Axis  Minor_Axis      Mag  Mag_Error       Flux  Flux_Error       CXX       CXY       CYY
498  2001ah  pantheon_data_folder/2001ah/mastDownload/HST/hst_16075_01_wfc3_ir_f110w_ie9801/hst_16075_01_wfc3_ir_f110w_ie9801_drz.fits  2024-11-09       HST       WFC3  F110W -0.000036  1.554060e-22 -3.690683e-22  0.000036  hst_16075_01_wfc3_ir_f110w_ie9801_drz.fits  hst_16075_01_wfc3_ir_f110w_ie9801_drz       403.0  185.728769   15.822338           52.44      47.347      38.992 -15.5041     0.0001  1590907.0    160.9632  0.000579 -0.000205  0.000525
Setting RA and Dec: 185.7287693, 15.8223383
RA and Dec: